In [34]:
import pandas as pd
pd.set_option('display.max_columns', None)

import sys
sys.path.append(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data_science_code')

from data_processor import DataProcessor
from classification_models import LogisticRegressionModel
from convert_categorical_variables import CategoricalEncoder
from evaluate_classification import ClassificationEvaluator

In [35]:
data_processor = DataProcessor()
train_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\train.csv')
test_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\test.csv')
gender_submission = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\gender_submission.csv')

In [36]:
train_data, test_data = data_processor.train_test_split(train_df, test_size=0.2, random_state=42)
X_train, y_train = data_processor.split_features_target(train_data, 'Survived')
X_test, y_test = data_processor.split_features_target(test_data, 'Survived')

Data split into training and testing sets successfully.
Features and target variable split successfully.
Features and target variable split successfully.


# EDA/ Feature Engineering

In [37]:
def eda(df_1):
    df = df_1.copy()
    df.drop('PassengerId', axis=1, inplace=True)

    df['Surname'] = df['Name'].str.split(',').str[0]
    df['FirstName'] = df['Name'].str.split(',').str[1]
    df['Title'] = df['FirstName'].str.split('.').str[0]
    df.drop(['Name', 'FirstName', 'Surname'], axis=1, inplace=True)
    rare_titles = ['Rev', 'Dr', 'Major', 'Col', 'Mlle', 'Capt', 'Mme', 'the Countess', 'Lady', 'Sir', 'Jonkheer', 'Don']
    df['Title'] = df['Title'].apply(lambda x: 'Miss' if x.strip() == 'Ms' else 'Rare' if x.strip() in rare_titles else x.strip())

    df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})

    df['Ticket_string'] = df['Ticket'].str.contains('[a-zA-Z]').astype(int)

    df['Cabin'] = df['Cabin'].fillna('U')

    most_common = df['Embarked'].mode()[0]
    df['Embarked'] = df['Embarked'].fillna(most_common)

    df['Age'] = df['Age'].fillna(df['Age'].median())

    df['Cabin_Letter'] = df['Cabin'].str.extract(r'([A-Za-z])')
    df['Cabin_Number'] = df['Cabin'].str.extract(r'(\d+)')
    df['Cabin_Number'] = df['Cabin_Number'].fillna(0)
    
    df = df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Title','Ticket_string','Cabin_Letter','Cabin_Number']]

    encoder = CategoricalEncoder(df)
    df = encoder.one_hot_encoding(['Embarked', 'Title', 'Cabin_Letter'])
    df = df.drop(['Cabin_Letter_U'], axis=1)

    return df

X_train_filt = eda(X_train)
X_test_filt = eda(X_test)

In [38]:
log_reg_model = LogisticRegressionModel()
log_reg_model.set_params(random_state=42, max_iter=1000) 
log_reg_model.train(X_train_filt, y_train)
print(log_reg_model.get_coefficients())
print(log_reg_model.get_params())


(array([[-0.69370142, -1.41078533, -0.02511666, -0.4737376 , -0.3552759 ,
         0.00438872,  0.1725667 ,  0.00480394,  0.21989625,  0.04530098,
        -0.26167186,  1.35841873, -0.18259521, -1.51752857,  0.99256796,
        -0.64733753, -0.07134182,  0.32167614, -0.23990107,  0.78798346,
         1.21398082,  0.47205742, -0.6552703 , -0.19161863]]), array([3.57653026]))
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
